In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pickle
import os


/var/folders/ld/s5_rrf_n32q9rltztl0sz_fh0000gn/T/ipykernel_13010/4138106253.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def train_and_save_knn_model():
    file_path = '/Users/bididudy/code/marcnaweb/car_recommendation_engine/raw_data/unique_car_models_df.csv'
    df = pd.read_csv(file_path)

    model_dir = '/Users/bididudy/code/marcnaweb/car_recommendation_engine/models'  # Folder name where models will be saved

    # Create the directory if it does not exist
    os.makedirs(model_dir, exist_ok=True)

    features_df = df.set_index('car_code', inplace=False)
    features_df = features_df.drop(columns=['car_model'])

    knn = NearestNeighbors(n_neighbors=6)
    knn.fit(features_df)

    # Save the kNN model to the 'models' directory
    with open(os.path.join(model_dir, 'knn_model.pkl'), 'wb') as f:
        pickle.dump(knn, f)

    # Save the preprocessed features DataFrame to the 'models' directory
    with open(os.path.join(model_dir, 'features_df.pkl'), 'wb') as f:
        pickle.dump(features_df, f)


In [4]:
# Call the function to train and save your model
train_and_save_knn_model()

In [20]:
def show_similar_cars(car_code):
    # Specify the directory where your models are saved
    model_dir = '/Users/bididudy/code/marcnaweb/car_recommendation_engine/models'  # Folder name where models are saved

    # Construct the path to each pickle file
    knn_model_path = os.path.join(model_dir, 'knn_model.pkl')
    features_df_path = os.path.join(model_dir, 'features_df.pkl')

    # Load the kNN model from the 'models' directory
    with open(knn_model_path, 'rb') as f:
        knn = pickle.load(f)

    # Load the preprocessed features DataFrame from the 'models' directory
    with open(features_df_path, 'rb') as f:
        features_df = pickle.load(f)

    # Use the car's features as our query point
    query = features_df.loc[car_code].values.reshape(1, -1)

    # Find the 5 nearest neighbors to the car
    distances, indices = knn.kneighbors(query)

    # Get the car codes of the 6 closest matches (including the query point itself)
    closest_cars_indices = indices.flatten()
    closest_cars = features_df.iloc[closest_cars_indices].reset_index()[['car_code']]

    # Assuming you need to join back with the original df to get car_model names
    df = pd.read_csv('/Users/bididudy/code/marcnaweb/car_recommendation_engine/raw_data/unique_car_models_df.csv')
    #closest_cars_with_models = pd.merge(closest_cars, df[['car_code', 'car_model']], on='car_code', how='left')
    closest_cars_with_models = pd.merge(closest_cars, df[['car_code', 'car_model']], left_index=True, right_on='car_code', how='left').set_index('car_code')

    return closest_cars_with_models


In [21]:
show_similar_cars(23794)

/Users/bididudy/.pyenv/versions/3.10.6/envs/car_recommendation_engine/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


TypeError: 'DataFrame' object is not callable

In [12]:
df.car_code

0    23794
1    13009
2    13010
3    13518
4     9609
5    19635
Name: car_code, dtype: int64

In [13]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [14]:
def show_similar_cars(car_code):
    '''

    '''
    file_path = '/Users/bididudy/code/marcnaweb/car_recommendation_engine/raw_data/unique_car_models_df.csv'
    df = pd.read_csv(file_path)

    features_df = df

    features_df.set_index('car_code', inplace=True)

    features_df = df.drop(columns=['car_model'])

    knn = NearestNeighbors(n_neighbors=6)

    knn.fit(features_df)

    # Use a car's features as our query point
    query = features_df.loc[car_code].values.reshape(1, -1)

    # Find the 5 nearest neighbors to the first car (excluding itself)
    distances, indices = knn.kneighbors(query)

    # Get the car codes of the 6 closest matches (including the query point itself)
    closest_cars_indices = indices.flatten()
    closest_cars = df.iloc[closest_cars_indices][['car_model']]

    return closest_cars